# 3D Droplet Oscillation

Results published: hopefully at some point!


### Preliminaries

This example can be found in the source code repository as as `ConfinedCylinder_ConvergenceStudy.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Initialization tasks

Loading the `XNSE_Solver` namespace:

In [34]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;

Initialization of the Workflow management; there `ConfinedCylinder_ConvergenceStudy` is the project name which is used for all comutations (aka. sessions):

In [3]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'C:\BoSSS-Databases\OscillatingDroplet3D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [4]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,Password,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]",,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\dc1\userspace\kummer\cluster,False,FDY-WindowsHPC,dotnet,,[ \\dc1\userspace\kummer == ],FDY\kummer,DC2,<null>,Normal,True,,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,W:\bosss_deploy,False,LbI,dotnet,,[ ],fk69umer,lcluster8.hrz.tu-darmstadt.de,,,,<null>,C:\Users\flori\.ssh\id_rsa,<null>,/home/fk69umer/bosss_deploy,project01287,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\bosss_deploy,False,Lb2-specialPrj,dotnet,,[ X: == /work/scratch/fk69umer ],fk69umer,lcluster16.hrz.tu-darmstadt.de,,,,<null>,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",/work/scratch/fk69umer/bosss_deploy,special00006,<null>,False


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [5]:
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]"


## Grid Creation

In [6]:
BoSSSshell.WorkflowMgm.Grids

IOException caught during creation/opening of database: The network path was not found. : '\\dc1\userspace\kummer\OscillatingDroplet3D'.
DirectoryNotFoundException caught during creation/opening of database: Could not find a part of the path 'X:\OscillatingDroplet3D'..


#0: { Guid = 67c6fd9e-293a-4c7a-805b-9fed5b5a64a3; Name = OscillatingDroplet3D_12x12x24_quarterDomain; Cell Count = 3456; Dim = 3 }
#1: { Guid = feee49dd-1f5a-4b67-9e04-364da4ac9872; Name = OscillatingDroplet3D_6x6x12_quarterDomain; Cell Count = 432; Dim = 3 }


### Quater-Domain grids
(Symmetry planes at $x = 0$ and $y = 0$)

In [26]:
int[] Resolutions = new int[] { 6, 12 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-3, 3, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}

Grid already found in database - identifid by name OscillatingDroplet3D_6x6x12_quarterDomain
Grid already found in database - identifid by name OscillatingDroplet3D_12x12x24_quarterDomain


In [10]:
wmg.Grids

#0: { Guid = 67c6fd9e-293a-4c7a-805b-9fed5b5a64a3; Name = OscillatingDroplet3D_12x12x24_quarterDomain; Cell Count = 3456; Dim = 3 }
#1: { Guid = feee49dd-1f5a-4b67-9e04-364da4ac9872; Name = OscillatingDroplet3D_6x6x12_quarterDomain; Cell Count = 432; Dim = 3 }


## Setup of control objects for a solver runs

In [29]:
List<XNSE_Control> Controls = new List<XNSE_Control>();

In [35]:
Controls.Clear();
int[] modeS = new int[] {2, 3, 4};
int[] aP_indiceS = new int[] { 0, 1, 2 };
bool[] useARM = new bool[] { false, true };

foreach(bool bARM in useARM) {
foreach(var grd in Grids) {
foreach(int mode in modeS) {
foreach(int k )
foreach(int aP_index in aP_indiceS) {

    double r0 = 0.001;
    var C = Droplet.OscillatingDroplet3D_LegendrePolynomials(r0, mode, aP_index);
    C.SetGrid(grd);
    C.SetDGdegree(3);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1260.0;
    C.PhysicalParameters.rho_B = 1.26;
    C.PhysicalParameters.mu_A = 0.0094;
    C.PhysicalParameters.mu_B = 9.4E-06;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 0.007;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.dtFixed = 5E-05;
    
    if(bARM) {
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = 1 }
        );
    }
    
    Controls.Add(C);
}
}
}
}

In [38]:
//Controls[0].InitialValues["Phi"]

In [48]:
(int)(Controls[0].VelocityBlockPrecondMode)

7

In [47]:
Controls[0].LevelSet_ConvergenceCriterion

1E-06

## Launch Jobs

In [10]:
Controls.Count

9

In [11]:
foreach(var ctrl in Controls) {
    //var oneJob              = ctrl.CreateJob();
    //oneJob.NumberOfMPIProcs = 1;
    //oneJob.Activate(myBatch); 
}

Empty job name - picking new name 'EmptyJobName_1'
Deploying job EmptyJobName_1 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060944
copied 49 files.
   written file: control.obj
deployment finished.
Starting mini batch processor in external process...
Started mini batch processor on local machine, process id is 20352.
started.

Empty job name - picking new name 'EmptyJobName_2'
Deploying job EmptyJobName_2 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060948
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Empty job name - picking new name 'EmptyJobName_3'
Deploying job EmptyJobName_3 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060948-1
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Empty job name - picking new name 'EmptyJobName_4'
Deploying job EmptyJobName_4 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060949
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Empty job name - picking new name 'EmptyJobName_5'
Deploying job EmptyJobName_5 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060949-1
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Empty job name - picking new name 'EmptyJobName_6'
Deploying job EmptyJobName_6 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060950
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Empty job name - picking new name 'EmptyJobName_7'
Deploying job EmptyJobName_7 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060951
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Empty job name - picking new name 'EmptyJobName_8'
Deploying job EmptyJobName_8 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060951-1
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.

Empty job name - picking new name 'EmptyJobName_9'
Deploying job EmptyJobName_9 ... 


Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\OscillatingDroplet3D-XNSE_Solver2021Oct27_060952
copied 49 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.



In [12]:
// wait for all jobs to finish (up to 5 days, check every 5 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*5), PollingIntervallSeconds:(60*5));

All jobs finished.


In [13]:
// detect failed Jobs in the job management
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects

Unhandled exception: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#16.<>c.<<Initialize>>b__0_1(Job job)
   at System.Linq.Enumerable.WhereEnumerableIterator`1.ToArray()
   at Submission#16.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [14]:
NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

Unhandled exception: System.ArgumentNullException: Value cannot be null. (Parameter 'source')
   at System.Linq.ThrowHelper.ThrowArgumentNullException(ExceptionArgument argument)
   at System.Linq.Enumerable.Count[TSource](IEnumerable`1 source)
   at Submission#17.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### Inspect the output of some arbitrary job:

In [15]:
BoSSSshell.WorkflowMgm.AllJobs.First().Value.Stdout

      ___           ___           ___           ___           ___     
     /\  \         /\  \         /\  \         /\  \         /\  \    
    /::\  \       /::\  \       /::\  \       /::\  \       /::\  \   
   /:/\:\  \     /:/\:\  \     /:/\ \  \     /:/\ \  \     /:/\ \  \  
  /::\~\:\__\   /:/  \:\  \   _\:\~\ \  \   _\:\~\ \  \   _\:\~\ \  \ 
 /:/\:\ \:|__| /:/__/ \:\__\ /\ \:\ \ \__\ /\ \:\ \ \__\ /\ \:\ \ \__\
 \:\~\:\/:/  / \:\  \ /:/  / \:\ \:\ \/__/ \:\ \:\ \/__/ \:\ \:\ \/__/
  \:\ \::/  /   \:\  /:/  /   \:\ \:\__\    \:\ \:\__\    \:\ \:\__\  
   \:\/:/  /     \:\/:/  /     \:\/:/  /     \:\/:/  /     \:\/:/  /  
    \::/__/       \::/  /       \::/  /       \::/  /       \::/  /   
     ~~            \/__/         \/__/         \/__/         \/__/    
                                                                      
Running with 1 MPI process (single core)
User: flori
Node: stormbreaker (rank 0)
arg #0 override from environment variable 'BOSSS_ARG_0': --control
